In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py as h5

sys.path.append('../../../common_code')

#Custom imports
# for color map
import seaborn as sns
# for reading datafiles 
import pandas as pd
import math

# import script that has formation channel classification functions:
from PostProcessingScripts import * 
from formation_channels import *  # downloads the functions to calculate formation channels 


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd

channelColorDict_lighter = {'classic':adjust_lightness(color='#118AB2', amount=1.6),'stable B no CEE':adjust_lightness(color='orange', amount=1.4), 'immediate CE':adjust_lightness(color='#EF476F', amount=1.2),\
                            r'double-core CE':adjust_lightness(color='#073B4C', amount=1.8), 'other':adjust_lightness(color='gray', amount=1.5),  'vi':adjust_lightness(color='cyan', amount=1.5), 'vii':adjust_lightness(color='#FFD166', amount=1.2)}

adjustedChannelList = ['classic', 'stable B no CEE', 'immediate CE',  r'double-core CE', 'other', 'vii', 'vi'] # channels to plot 




### test to check if the formation channels were correctly added to print(fdata["formationchannel_z_rates"]['fraction_classic_000'][()]) 
See below, this works: the different CE classes perfectly add up between the formation channels to form the other 
So something must have gone wrong when adding it to the CSV files (probably a header issue) --> Checking this now

In [17]:
def obtain_redshiftsruns(pathData = '/Volumes/SimonsFoundation/DataDCO/'):
    BPSmodelName='A'
    DCOtype='BNS'
    path_ = '/Volumes/SimonsFoundation/DataDCO/' + alphabetDirDict[BPSmodelName] +'/'
    path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
    fdata = h5.File(path, 'r')
    redshifts = fdata['redshifts']['redshift'][...].squeeze()
    fdata.close()
    return redshifts 

##### PLOT LVKM1 

pathData='/Volumes/SimonsFoundation/DataDCO/'
redshifts = redshifts_runs = obtain_redshiftsruns(pathData = pathData)
print(redshifts)



[0.19230769 0.57692308 0.96153846 1.34615385 1.73076923 2.11538462
 2.5        2.88461538 3.26923077 3.65384615 4.03846154 4.42307692
 4.80769231 5.19230769 5.57692308 5.96153846 6.34615385 6.73076923
 7.11538462 7.5        7.88461538 8.26923077 8.65384615 9.03846154
 9.42307692 9.80769231]


In [32]:
DCOtype='BNS'
BPSmodelName='D'

DCOname = DCOname_dict[DCOtype]
path  = pathData + alphabetDirDict[BPSmodelName] +'/COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'

fdata = h5.File(path, 'r')


print(fdata["formationchannel_z_rates"]['fraction_classic_000'][()])
for ind_mssfr, mssfr in enumerate(MSSFRnameslist):
    
    other_fc =  fdata["formationchannel_z_rates"]['fraction_other_%s'%(mssfr)][0]
    
    other_split = 0
    for fc_header in ['other_withCE', 'other_withoutCE']:
    
        header_mssfr = 'fraction_%s_%s'%(fc_header, mssfr)
    
        other_split+=fdata["formationchannel_z_rates"][header_mssfr][0]
    
    print(mssfr, ' difference is ', other_fc-other_split)
        
        
    print()
    
    
fdata.close()

[0.68049381 0.73812531 0.79972212 0.83075927 0.83057015 0.79198572
 0.68079014 0.46143449 0.23699463 0.1302508  0.09497424 0.08278633
 0.08032815 0.08408962 0.0997029  0.13026007 0.17514809 0.22797473
 0.2791893  0.32188681 0.35033353 0.35797499 0.33842964 0.29760524
 0.0686072  0.        ]
000  difference is  1.734723475976807e-18

111  difference is  -1.734723475976807e-18

112  difference is  0.0

113  difference is  0.0

121  difference is  -1.734723475976807e-18

122  difference is  0.0

123  difference is  0.0

131  difference is  -3.469446951953614e-18

132  difference is  -1.734723475976807e-18

133  difference is  1.734723475976807e-18

211  difference is  3.469446951953614e-18

212  difference is  0.0

213  difference is  1.734723475976807e-18

221  difference is  -3.469446951953614e-18

222  difference is  -1.734723475976807e-18

223  difference is  1.734723475976807e-18

231  difference is  0.0

232  difference is  3.469446951953614e-18

233  difference is  1.73472347597680

### now checking the CVS creation. Because this should just use the info above, but somehow is not? 

In [9]:
fdata.close()

In [8]:
DCOname='NSNS'
DCOtype='BNS'
stringgg =  'Formation_Channels_Local_Rates'
pathData='/Volumes/SimonsFoundation/DataDCO/'
which_z_ind=0

headerDict_intrinsicOther = {'other_withCE':'channel V with CE intrinsic (z=0)', 'other_withoutCE':'channel V without CE intrinsic (z=0)'}

for MSSFRname in MSSFRnameslist:
    csv_file_path = '/Users/floorbroekgaarden/Projects/GitHub/DCO_FormationChannels/dataFiles/data_Fig_1/'  + stringgg + '_'  + DCOname + '_xyz_' + MSSFRname + '.csv' 
    df = pd.read_csv(csv_file_path)
    
    
    
    ### get the values
    for ind_c, whichChannel in enumerate(['other_withCE', 'other_withoutCE']):    
        merger_ratio_z = np.zeros(len(BPSnameslist))
        for ind_m, BPSmodelName in enumerate(BPSnameslist):
            
            full_data_path  = pathData + alphabetDirDict[BPSmodelName] +'/COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
            fparam_key = "formationchannel_z_rates"
            
            header = "fraction_" + whichChannel + "_" + MSSFRname
            
            fdata = h5.File(full_data_path,'r')
            print(fdata["formationchannel_z_rates"].keys())
            fc_fraction = fdata[fparam_key][header][...].squeeze()[which_z_ind]  # [which_z_ind = 0] gives at lowest redshift
            merger_ratio_z[ind_m] = fc_fraction
            fdata.close()

            
            
        # Step 4: Write the DataFrame back to the CSV file
        column_name = headerDict_intrinsicOther[whichChannel] 
        df[column_name] = merger_ratio_z
        df.to_csv(csv_file_path, index=False)


<KeysViewHDF5 ['fraction_classic_000', 'fraction_classic_111', 'fraction_classic_112', 'fraction_classic_113', 'fraction_classic_121', 'fraction_classic_122', 'fraction_classic_123', 'fraction_classic_131', 'fraction_classic_132', 'fraction_classic_133', 'fraction_classic_211', 'fraction_classic_212', 'fraction_classic_213', 'fraction_classic_221', 'fraction_classic_222', 'fraction_classic_223', 'fraction_classic_231', 'fraction_classic_232', 'fraction_classic_233', 'fraction_classic_311', 'fraction_classic_312', 'fraction_classic_313', 'fraction_classic_321', 'fraction_classic_322', 'fraction_classic_323', 'fraction_classic_331', 'fraction_classic_332', 'fraction_classic_333', 'fraction_double-core CE_000', 'fraction_double-core CE_111', 'fraction_double-core CE_112', 'fraction_double-core CE_113', 'fraction_double-core CE_121', 'fraction_double-core CE_122', 'fraction_double-core CE_123', 'fraction_double-core CE_131', 'fraction_double-core CE_132', 'fraction_double-core CE_133', 'fr

KeyError: "Unable to open object (object 'fraction_other_withCE_000' doesn't exist)"

[0.19230769 0.57692308 0.96153846 1.34615385 1.73076923 2.11538462
 2.5        2.88461538 3.26923077 3.65384615 4.03846154 4.42307692
 4.80769231 5.19230769 5.57692308 5.96153846 6.34615385 6.73076923
 7.11538462 7.5        7.88461538 8.26923077 8.65384615 9.03846154
 9.42307692 9.80769231]


In [ ]:
# BPSmodelName='A'
# DCOtype='BHNS'
# path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
# fdata = h5.File(path, 'r') # read in data
# print(fdata['formationchannel_z_rates']['fraction_other_000'][()])

In [ ]:
# whichChannel [0.04491331 0.0381057  0.04297189 0.04365361 0.04396137 0.04309813
#  0.04012219 0.03583868 0.03081875 0.02474782 0.01893365 0.01483051
#  0.01252746 0.01102444 0.01139272 0.01169556 0.01067282 0.01144997
#  0.01994581 0.02413152 0.02541153 0.02077036 0.01541305 0.01312366
#  0.01288913 0.00272771]

In [10]:
# DCOtype='BNS'

# # for ind_bps, BPSmodelName in enumerate(BPSnameslist):
# for ind_bps, BPSmodelName in enumerate([BPSnameslist[0]]):
#     path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5' # path to data 
#     fdata = h5.File(path, 'a')
    
#     print(fdata['formationchannel_z_rates'].keys())


#     fdco = fdata['doubleCompactObjects']
#     channels = fdco['formaton channel'][...].squeeze()
#     # get other channel 
#     Channel = 'other'
#     ind_wanted = dictFormationChannelIndex[Channel]
#     CEflag = fdco['doubleCommonEnvelopeFlag'][...].squeeze()

    
#     for whichChannel in ['other_withCE', 'other_withoutCE']:
        
#         if whichChannel=='other_withCE': mask_fc = (channels==ind_wanted) & (CEflag==1)  # other & CE experience
#         elif whichChannel=='other_withoutCE': mask_fc = (channels==ind_wanted) & (CEflag==0)  # does not experience a CE
        
        
#         for ind_mssfr, mssfr in enumerate(MSSFRnameslist[:]):
# #         for ind_mssfr, mssfr in enumerate([MSSFRnameslist[0]]):
#             print('mssfr ', mssfr)
#             if np.sum(mask_fc)>1:
#                 fparam_key = 'weights_intrinsicPerRedshift'
#                 fractions_z = np.zeros_like(redshifts)
#                 for ind_z, redshift in enumerate(redshifts):
#                     redshift = np.round(redshift,4)
#                     weightheader = 'w_' + mssfr + '_z_' +  str(redshift)
#                     weights_ = fdata[fparam_key][weightheader][...].squeeze()  
#                     if np.sum(weights_)!=0:
#                         fractions_z[ind_z] = np.sum(weights_[mask_fc])/np.sum(weights_)
#                     del weights_
                        
#             else:
#                 fractions_z = np.zeros_like(redshifts)
                
                
            
#         print(whichChannel, fractions_z)
        
#         fc_header_name = "fraction_" + whichChannel + "_" + mssfr 
#         channel_no = fdata["formationchannel_z_rates"].create_dataset(u"%s"%fc_header_name, data=fractions_z)
#         channel_no.attrs[u"units"] = u"#"   
        
        


#     fdata.close()


BlockingIOError: [Errno 35] Unable to open file (unable to lock file, errno = 35, error message = 'Resource temporarily unavailable')

In [ ]:
# DCOtype='BBH'

# for ind_bps, BPSmodelName in enumerate(BPSnameslist):
# # for ind_bps, BPSmodelName in enumerate([BPSnameslist[0]]):
#     path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5' # path to data 
#     fdata = h5.File(path, 'a')
    
#     print(BPSmodelName)


#     fdco = fdata['doubleCompactObjects']
#     channels = fdco['formaton channel'][...].squeeze()
#     # get other channel 
#     Channel = 'other'
#     ind_wanted = dictFormationChannelIndex[Channel]
#     CEflag = fdco['doubleCommonEnvelopeFlag'][...].squeeze()

    
#     for whichChannel in ['other_withCE', 'other_withoutCE']:
        
#         if whichChannel=='other_withCE': mask_fc = (channels==ind_wanted) & (CEflag==1)  # other & CE experience
#         elif whichChannel=='other_withoutCE': mask_fc = (channels==ind_wanted) & (CEflag==0)  # does not experience a CE
        
        
#         for ind_mssfr, mssfr in enumerate(MSSFRnameslist[:]):
# #         for ind_mssfr, mssfr in enumerate([MSSFRnameslist[0]]):
#             print('mssfr ', mssfr)
#             if np.sum(mask_fc)>1:
#                 fparam_key = 'weights_intrinsicPerRedshift'
#                 fractions_z = np.zeros_like(redshifts)
#                 for ind_z, redshift in enumerate(redshifts):
#                     redshift = np.round(redshift,4)
#                     weightheader = 'w_' + mssfr + '_z_' +  str(redshift)
#                     weights_ = fdata[fparam_key][weightheader][...].squeeze()  
#                     if np.sum(weights_)!=0:
#                         fractions_z[ind_z] = np.sum(weights_[mask_fc])/np.sum(weights_)
#                     del weights_
                        
#             else:
#                 fractions_z = np.zeros_like(redshifts)
                
                
            
#         print(whichChannel, fractions_z)
        
#         fc_header_name = "fraction_" + whichChannel + "_" + mssfr 
#         channel_no = fdata["formationchannel_z_rates"].create_dataset(u"%s"%fc_header_name, data=fractions_z)
#         channel_no.attrs[u"units"] = u"#"   
        
        


#     fdata.close()


A
mssfr  000
mssfr  111
mssfr  112
mssfr  113
mssfr  121
mssfr  122
mssfr  123
mssfr  131
mssfr  132
mssfr  133
mssfr  211
mssfr  212
mssfr  213
mssfr  221
mssfr  222
mssfr  223
mssfr  231
mssfr  232
mssfr  233
mssfr  311
mssfr  312
mssfr  313
mssfr  321
mssfr  322
mssfr  323
mssfr  331
mssfr  332
mssfr  333
other_withCE [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
mssfr  000
mssfr  111
mssfr  112
mssfr  113
mssfr  121
mssfr  122
mssfr  123
mssfr  131
mssfr  132
mssfr  133
mssfr  211
mssfr  212
mssfr  213
mssfr  221
mssfr  222
mssfr  223
mssfr  231
mssfr  232
mssfr  233
mssfr  311
mssfr  312
mssfr  313
mssfr  321
mssfr  322
mssfr  323
mssfr  331
mssfr  332
mssfr  333
other_withoutCE [0.03115116 0.0336064  0.0335415  0.03313544 0.0320504  0.03025586
 0.02873787 0.02774184 0.02726086 0.02776589 0.02922352 0.03155204
 0.03470512 0.03862893 0.04338852 0.04884217 0.05480367 0.06139899
 0.06855064 0.0761     0.08423156 0.0930998  0.10250574 0.11332571
 0.1

I
mssfr  000
mssfr  111
mssfr  112
mssfr  113
mssfr  121
mssfr  122
mssfr  123
mssfr  131
mssfr  132
mssfr  133
mssfr  211
mssfr  212
mssfr  213
mssfr  221
mssfr  222
mssfr  223
mssfr  231
mssfr  232
mssfr  233
mssfr  311
mssfr  312
mssfr  313
mssfr  321
mssfr  322
mssfr  323
mssfr  331
mssfr  332
mssfr  333
other_withCE [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
mssfr  000
mssfr  111
mssfr  112
mssfr  113
mssfr  121
mssfr  122
mssfr  123
mssfr  131
mssfr  132
mssfr  133
mssfr  211
mssfr  212
mssfr  213


In [ ]:
print(fdata['formationchannel_z_rates']['fraction_classic_000'][()])

In [ ]:
fdata.close()

In [15]:
# DCOname='BHBH'
# DCOtype='BBH'

# DCOname='BHNS'
# DCOtype='BHNS'


DCOname='NSNS'
DCOtype='BNS'

stringgg =  'Formation_Channels_Local_Rates'
pathData='/Volumes/SimonsFoundation/DataDCO/'
which_z_ind=0

headerDict_intrinsicOther = {'other_withCE':'channel V with CE intrinsic (z=0)', 'other_withoutCE':'channel V without CE intrinsic (z=0)'}

for MSSFR in MSSFRnameslist:
    csv_file_path = '/Users/floorbroekgaarden/Projects/GitHub/DCO_FormationChannels/dataFiles/data_Fig_1/'  + stringgg + '_'  + DCOname + '_xyz_' + MSSFR + '.csv' 
    df = pd.read_csv(csv_file_path)
    
    
    
    ### get the values
    for ind_c, whichChannel in enumerate(['other_withCE', 'other_withoutCE']):    
        merger_ratio_z = np.zeros(len(BPSnameslist))
        for ind_m, BPSmodelName in enumerate(BPSnameslist):
            print(BPSmodelName)
            
            full_data_path  = pathData + alphabetDirDict[BPSmodelName] +'/COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
            fparam_key = "formationchannel_z_rates"
            header = "fraction_" + whichChannel + "_" + MSSFRname
            
            fdata = h5.File(full_data_path,'r')
            fc_fraction = fdata[fparam_key][header][...].squeeze()[which_z_ind]  # [which_z_ind = 0] gives at lowest redshift
            merger_ratio_z[ind_m] = fc_fraction
            fdata.close()

            
            
        # Step 4: Write the DataFrame back to the CSV file
        column_name = headerDict_intrinsicOther[whichChannel] 
        df[column_name] = merger_ratio_z
        df.to_csv(csv_file_path, index=False)


A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T


In [ ]:
csv_file_path = 'mon.csv'
df = pd.read_csv(csv_file_path)

# Step 2: Define the values for the new &quot;City&quot; column
new_city_values = ['New York', 'Los Angeles', 'Chicago', 'San Francisco']

# Step 3: Add the new &quot;City&quot; column to the DataFrame
df['City'] = new_city_values

# Step 4: Write the DataFrame back to the CSV file
df.to_csv(csv_file_path, index=False)

In [ ]:
DCOtype='BHNS'

for ind_bps, BPSmodelName in enumerate(BPSnameslist):
# for ind_bps, BPSmodelName in enumerate([BPSnameslist[0]]):
    path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5' # path to data 
    fdata = h5.File(path, 'a')
    
    print(fdata['formationchannel_z_rates'].keys())


    fdco = fdata['doubleCompactObjects']
    channels = fdco['formaton channel'][...].squeeze()
    # get other channel 
    Channel = 'other'
    ind_wanted = dictFormationChannelIndex[Channel]
    CEflag = fdco['doubleCommonEnvelopeFlag'][...].squeeze()

    
    for whichChannel in ['other_withCE', 'other_withoutCE']:
        
        if whichChannel=='other_withCE': mask_fc = (channels==ind_wanted) & (CEflag==1)  # other & CE experience
        elif whichChannel=='other_withoutCE': mask_fc = (channels==ind_wanted) & (CEflag==0)  # does not experience a CE
        
        
        for ind_mssfr, mssfr in enumerate(MSSFRnameslist[:]):
#         for ind_mssfr, mssfr in enumerate([MSSFRnameslist[0]]):
            print('mssfr ', mssfr)
            if np.sum(mask_fc)>1:
                fparam_key = 'weights_intrinsicPerRedshift'
                fractions_z = np.zeros_like(redshifts)
                for ind_z, redshift in enumerate(redshifts):
                    redshift = np.round(redshift,4)
                    weightheader = 'w_' + mssfr + '_z_' +  str(redshift)
                    weights_ = fdata[fparam_key][weightheader][...].squeeze()  
                    if np.sum(weights_)!=0:
                        fractions_z[ind_z] = np.sum(weights_[mask_fc])/np.sum(weights_)
                    del weights_
                        
            else:
                fractions_z = np.zeros_like(redshifts)
                
                
            
        print(whichChannel, fractions_z)
        
        fc_header_name = "fraction_" + whichChannel + "_" + mssfr 
        channel_no = fdata["formationchannel_z_rates"].create_dataset(u"%s"%fc_header_name, data=fractions_z)
        channel_no.attrs[u"units"] = u"#"   
        
        


    fdata.close()


In [ ]:


BPSmodelName='A'
DCOtype='BHNS'
path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'
fdata = h5.File(path, 'r') # read in data
print(fdata['formationchannel_z_rates']['fraction_other_000'][()])
print(fdata['formationchannel_z_rates']['other_withoutCE_000'][()])
print(fdata['formationchannel_z_rates']['other_withCE_000'][()])
fdata.close()


In [ ]:
print(adjustedChannelList)

In [ ]:
pathData='/Volumes/SimonsFoundation/DataDCO/'
BPSmodelName = 'A'
DCOtype='BHNS'


path  = pathData + alphabetDirDict[BPSmodelName] +'/' + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5' # path to data 
fdata = h5.File(path, 'r') # read in data 


fdco = fdata['doubleCompactObjects']

channels = fdco['formaton channel'][...].squeeze()

# other channel 
Channel = 'other'
ind_wanted = dictFormationChannelIndex[Channel]
c_FC = channelColorDict[Channel] # color of the channel 
mask_fc = (channels==ind_wanted) # get a mask for all the DCO systems that fulfill this criteria 

In [ ]:

print(fdco.keys())
CEflag = fdco['doubleCommonEnvelopeFlag'][...].squeeze()
CEmask = (mask_fc==1) & (CEflag==1) # other & CE


Nother = np.sum(mask_fc)
Nother_CE = np.sum(CEmask)

print(Nother, ' other fc out of ', len(mask_fc))
print('of which %s experience CE'%Nother_CE)



In [ ]:
print(fdata.keys())

In [ ]:
print(fdata['formationchannel_z_rates'].keys())

In [ ]:
print(fdata['formationchannel_z_rates']['fraction_classic_000'][()])

In [ ]:
fdata.close()




In [ ]:
def add_formation_channel_H5file(pathToData, DCOtype, BPSmodelName, redshifts):

    DCOname = DCOname_dict[DCOtype]

    # path for files 
    path  = pathToData + alphabetDirDict[BPSmodelName] +'/COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'

    # read in data 
    fdata = h5.File(path, 'r+')

    # delete the group if it already exists (so that we overwrite it)
    # if fdata.get("formationchannel_z_rates"):
    #     del f['dataset_name']
    if "formationchannel_z_rates" in fdata.keys():
        print("formation channel rates group already exists... deleting it now")
        del fdata["formationchannel_z_rates"]

    Rfc = fdata.create_group("formationchannel_z_rates")

    fDCO = fdata['doubleCompactObjects']
    channels = fdata['doubleCompactObjects']["formaton channel"][...].squeeze()
    # channels = fdata['doubleCompactObjects']['formaton channel'][...].squeeze()

    # add a line with the redshifts 
    channel_no = fdata["formationchannel_z_rates"].create_dataset(u"redshift", data=redshifts)
    channel_no.attrs[u"units"] = u"#"    



    # add all 
    # for ind_mssfr, mssfr in enumerate(['312']): # temp fix
    for ind_mssfr, mssfr in enumerate(MSSFRnameslist[:]):
        print('adding all for mssfr ', mssfr)
        # print(fdata.keys())
        fparam_key = 'weights_intrinsicPerRedshift'
        fractions_z = np.zeros_like(redshifts)
        for ind_z, redshift in enumerate(redshifts):
            redshift = np.round(redshift,4)
            weightheader = 'w_' + mssfr + '_z_' +  str(redshift)
            weights_ = fdata[fparam_key][weightheader][...].squeeze()  
            if np.sum(weights_)!=0:
                fractions_z[ind_z] = np.sum(weights_)
            del weights_
            # else:
            #     fractions_z = np.zeros_like(redshifts)

        # append total observed rate (integrated over redshift horizon LIGO/VIRGO/KAGRA)
        # Rdetxyz = Rdet.create_dataset(u"w_%s"%mssfrName, data=weightsDet)
        # Rdetxyz.attrs[u"units"] = u"yr^{-1}"
        # append total intrinsic rate at redshift 0 
        fc_header_name = "rate_" + "total" + "_" + mssfr 
        channel_no = fdata["formationchannel_z_rates"].create_dataset(u"%s"%fc_header_name, data=fractions_z)
        channel_no.attrs[u"units"] = u"Gpc^-3 yr^-1"  
        del fractions_z
    
    for nrC, Channel in enumerate(channelList): 
        fc_ind_wanted = dictFormationChannelIndex[Channel]
        mask_fc = (channels==fc_ind_wanted)
        # del channels
        for ind_mssfr, mssfr in enumerate(MSSFRnameslist[:]):
            print('mssfr ', mssfr)
            if np.sum(mask_fc)>1:
                fparam_key = 'weights_intrinsicPerRedshift'
                fractions_z = np.zeros_like(redshifts)
                for ind_z, redshift in enumerate(redshifts):
                    redshift = np.round(redshift,4)
                    weightheader = 'w_' + mssfr + '_z_' +  str(redshift)
                    weights_ = fdata[fparam_key][weightheader][...].squeeze()  
                    if np.sum(weights_)!=0:
                        fractions_z[ind_z] = np.sum(weights_[mask_fc])/np.sum(weights_)
                    del weights_
                        
            else:
                fractions_z = np.zeros_like(redshifts)

            # append total observed rate (integrated over redshift horizon LIGO/VIRGO/KAGRA)
            # Rdetxyz = Rdet.create_dataset(u"w_%s"%mssfrName, data=weightsDet)
            # Rdetxyz.attrs[u"units"] = u"yr^{-1}"
            # append total intrinsic rate at redshift 0 
            fc_header_name = "fraction_" + Channel + "_" + mssfr 
            channel_no = fdata["formationchannel_z_rates"].create_dataset(u"%s"%fc_header_name, data=fractions_z)
            channel_no.attrs[u"units"] = u"#"    
